# Defining Alleles v02

Based on original code by Benjamin Schwessinger.

#### Inputs
* Assemblytics alignment of all haplotigs onto their corresponding primary contigs. The *.oriented_coords.csv will be converted to a gff file of haplotig alignments onto their respective primary contigs
* gff file of all features. This will be filtered down to genes only and separated out for each contig
* protein and gene *fasta* files for reciprocal BLAST of haplotig sequences onto primary sequences
* the .poff from proteinortho.

#### Process
* for each primary gene/protein the overlapping haplotig is pulled out and added to the BLAST DataFrame
* best BLAST hits are first filtered on e-value minimum followed by BitScore maximum

* proteinortho was also used to identify alleles. This was run as follows:
```
/home/gamran/anaconda3/proteinortho_v5.16b/proteinortho5.pl -project=ph_ctg_516 -synteny ph_ctg_516/Pst_104E_v13_h_ctg.protein.faa ph_ctg_516/Pst_104E_v13_p_ctg.protein.faa
```
* Use the poff-graph and poff-proteinortho-graph for analysis
* Depending on the run, the poff_header needs to be defined. In the current case, 'Target' followed 'Query'

#### Outputs
* *Pst_104E_v13_p_ctg.full_df.alleles*: contains all proteinortho hits and the best BLAST (p on h) pairs out of those primary proteins not identified as having an allele pair by proteinortho.
* *Pst_104E_v13_h_ctg.full_df.alleles*: contains all the best BLAST (h on p) hits (h on p).
  * NOTE that initial allele cutoffs (QCov/TCov/PctID) are defined in `Pst_104E_post_allele_analysis_v02_RT`, and therefore the initial set of alleles is not defined in this notebook. Since we only want the best reciprocal BLAST hits for haplotig proteins that were not already identified as alleles, the filtering for removing haplotig proteins that have already been paired with a primary protein must occur in `Pst_104E_post_allele_analysis_v02_RT`.
* These files can be filtered based on QCov/TCov/%ID/Levenshtein similarity in `Pst_104E_post_allele_analysis_v02_RTs`.

In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import os
import re
from Bio import SeqIO
import pysam
from Bio.SeqRecord import SeqRecord
from pybedtools import BedTool
import numpy as np
import pybedtools
import time
import matplotlib.pyplot as plt
import sys
import subprocess
import shutil


In [3]:
#Define the PATH

GENOME_VERSION = 'Pst_104E_v12'

BASE_OUT_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/allele_analysis/defining_alleles' 
GENOME_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly'
ASSEMBLYTICS_IN_PATH = '/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Assembletics' 

BLAST_DB = os.path.join(BASE_OUT_PATH, 'blast_DB')
OUT_PATH = os.path.join(BASE_OUT_PATH, 'allele_analysis')
OUT_tmp = os.path.join(OUT_PATH, 'tmp')
PROTEINORTHO_OUT_PATH = os.path.join(BASE_OUT_PATH, 'proteinortho')
    
#renamed the allele path to reflect the proteinortho results
ALLELE_PATH = os.path.join(OUT_PATH, 'alleles_proteinortho_graph516')
if not os.path.exists(BASE_OUT_PATH):
    os.mkdir(BASE_OUT_PATH)
if not os.path.isdir(OUT_PATH):
    os.mkdir(OUT_PATH)
if not os.path.isdir(OUT_tmp):
    os.mkdir(OUT_tmp)
if not os.path.exists(ALLELE_PATH):
    os.mkdir(ALLELE_PATH)
if not os.path.exists(BLAST_DB):
    os.mkdir(BLAST_DB)
if not os.path.exists(PROTEINORTHO_OUT_PATH):
    os.mkdir(PROTEINORTHO_OUT_PATH)

#Define your p and h genome and move it into the allele analysis folder
P_GENOME = 'Pst_104E_v12_p_ctg'
H_GENOME = 'Pst_104E_v12_h_ctg'
for x in (x + '.fa' for x in [P_GENOME, H_GENOME]):
    shutil.copy2(GENOME_PATH+'/'+x, OUT_tmp)

In [4]:
#get proteinortho synteny file which ends with .poff
poff_graph_fn = os.path.join(PROTEINORTHO_OUT_PATH, 'ph_ctg_516.poff-graph')

In [5]:
#Define ENV parameters for blast hits and threads used in blast analysis
n_threads = 16
e_value = 1e-3

In [6]:
def att_column_p(x, y, z, a, b):
    '''Generate attribute column of h on p p_gff dataframe.
    '''
    string = 'h_contig=%s;query_start=%s;query_stop=%s;query_length=%s;query_aln_ln=%s' % (x, str(y), str(z), str(a), str(b))
    return string

def att_column_h(x, y, z, a, b):
    '''Generate attribute column of h on p h_gff dataframe.
    '''
    string = 'p_contig=%s;ref_start=%s;ref_stop=%s;ref_length=%s;ref_aln_ln=%s' % (x, str(y), str(z), str(a), str(b))
    return string

def gene_contig_subset(df, contig, feature):
    '''Input the gff and subset the feature columne by gene.
        Return data frame subset by contig and gene'''
    tmp_df = df[(df[0].str.contains(contig)) & (df[2]==feature)]
    tmp_df.sort_values(3,inplace=True)
    tmp_df.reset_index(drop=True, inplace=True)
    return tmp_df

def same_contig_blast(x,y):
    '''Function that checks if the blast hit in column y is on the same contig as the the query sequence in
    column y.
    '''
    q_contig = re.search(r'[p|h]([a-z]*_[0-9]*)_?', x).group(1)
    if y != 'NaN':
        hit_contig = re.search(r'[p|h]([a-z]*_[0-9]*)_?', y).group(1)
    else:
        hit_contig = 'NaN'
    return q_contig == hit_contig

def target_on_mapped_haplotig(t_contig, h_contig_overlap):
    '''Simple function that checks if the target contig is in the list of h overlapping contigs'''
    if t_contig == False or h_contig_overlap == False:
        return False
    return t_contig in h_contig_overlap

#### Considerations for blast analysis
Do gene blast and protein blast. Initially go off protein blast results. They should be more imformative as it also includes the coding region and the frame.
> Pull this in as a dataframe and filter it down as well to each contig. Do blasts both ways as well.
> Copy over primary & haplotig .protein.fa and .gene.fa files from the BASE_A_folder, if not already present.

In [7]:
#generate the blast databases if not already present
os.chdir(BLAST_DB)
# copy over .protein.fa and .gene.fa files from the GENOME_PATH folder
# if not already present, for both primary and haplotigs
shutil.copy2(os.path.join(GENOME_PATH, P_GENOME + '.anno.protein.fa'), BLAST_DB)
shutil.copy2(os.path.join(GENOME_PATH, P_GENOME + '.anno.gene.fa'), BLAST_DB)
shutil.copy2(os.path.join(GENOME_PATH, H_GENOME + '.anno.protein.fa'), BLAST_DB)
shutil.copy2(os.path.join(GENOME_PATH, H_GENOME + '.anno.gene.fa'), BLAST_DB)

blast_dir_content = os.listdir(BLAST_DB)
for x in blast_dir_content:
    if x.endswith('.fa') and ({os.path.isfile(x + e) for e in ['.psq', '.phr', '.pin'] } != {True}\
           and {os.path.isfile(x + e) for e in ['.nin', '.nhr', '.nsq'] } != {True} ):

        make_DB_options = ['-in']
        make_DB_options.append(x)
        make_DB_options.append('-dbtype')
        if 'protein' in x:
            make_DB_options.append('prot')
        else:
            make_DB_options.append('nucl')
        make_DB_command = 'makeblastdb %s' % ' '.join(make_DB_options)
        make_DB_stderr = subprocess.check_output(make_DB_command, shell=True, stderr=subprocess.STDOUT)
        print('%s is done!' % make_DB_command)
print("All databases generated and ready to go!")

All databases generated and ready to go!


In [8]:
blast_dict = {}
blast_dict['gene_fa'] = [x for x  in os.listdir(BLAST_DB) if x.endswith('gene.fa') and 'ph_ctg' not in x]
blast_dict['protein_fa'] = [x for x  in os.listdir(BLAST_DB) if x.endswith('protein.fa') and 'ph_ctg' not in x]
blast_stderr_dict = {}

for key in blast_dict.keys():
    for n, query in enumerate(blast_dict[key]):
        tmp_list = blast_dict[key][:]
        tmp_stderr_list = []
        #this loops through all remaining files and does blast against all those
        del tmp_list[n]
        for db in tmp_list:
            print("\nBlasting %s ..." %db)
            blast_options = ['-query']
            blast_options.append(query)
            blast_options.append('-db')

            blast_options.append(db)
            blast_options.append('-outfmt 6')
            blast_options.append('-evalue')
            blast_options.append(str(e_value))
            blast_options.append('-num_threads')
            blast_options.append(str(n_threads))
            blast_options.append('>')
            if 'gene' in query and 'gene' in db:
                out_name_list = [ query.split('.')[0], db.split('.')[0], str(e_value), 'blastn.outfmt6']
                out_name = os.path.join(OUT_PATH,'.'.join(out_name_list))
                blast_options.append(out_name)
                blast_command = 'blastn %s' % ' '.join(blast_options)
            elif 'protein' in query and 'protein' in db:
                out_name_list = [ query.split('.')[0], db.split('.')[0], str(e_value), 'blastp.outfmt6']
                out_name = os.path.join(OUT_PATH,'.'.join(out_name_list))
                blast_options.append(out_name)
                blast_command = 'blastp %s' % ' '.join(blast_options)
            print("Blast command generated:\n%s" % blast_command)
            if not os.path.exists(out_name):
                print("Executing new blast command...")
                blast_stderr_dict[blast_command] = subprocess.check_output(blast_command, shell=True, stderr=subprocess.STDOUT)
                print("New blast executed!")
            else:
                blast_stderr_dict[blast_command] = 'Previously done already!'
                print("Blast command completed previously!")


Blasting Pst_104E_v12_h_ctg.anno.protein.fa ...
Blast command generated:
blastp -query Pst_104E_v12_p_ctg.anno.protein.fa -db Pst_104E_v12_h_ctg.anno.protein.fa -outfmt 6 -evalue 0.001 -num_threads 16 > /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/allele_analysis/defining_alleles/allele_analysis/Pst_104E_v12_p_ctg.Pst_104E_v12_h_ctg.0.001.blastp.outfmt6
Blast command completed previously!

Blasting Pst_104E_v12_p_ctg.anno.protein.fa ...
Blast command generated:
blastp -query Pst_104E_v12_h_ctg.anno.protein.fa -db Pst_104E_v12_p_ctg.anno.protein.fa -outfmt 6 -evalue 0.001 -num_threads 16 > /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/allele_analysis/defining_alleles/allele_analysis/Pst_104E_v12_h_ctg.Pst_104E_v12_p_ctg.0.001.blastp.outfmt6
Blast command completed previously!

Blasting Pst_104E_v12_h_ctg.anno.gene.fa ...
Blast command generated:
blastn -query Pst_104E_v12_p_ctg.anno.gene.fa -

In [9]:
# make a sequence length dict for all genes and proteins for which blast was performed
seq_list = []
length_list = []
for key in blast_dict.keys():
    for file in blast_dict[key]:
        for seq in SeqIO.parse(open(os.path.join(GENOME_PATH, file)), 'fasta'):
            seq_list.append(seq.id)
            length_list.append(len(seq.seq))
length_dict = dict(zip(seq_list, length_list))

In [10]:
# get assemblytics folders
ass_folders = [os.path.join(ASSEMBLYTICS_IN_PATH, x) for x in os.listdir(ASSEMBLYTICS_IN_PATH) if x.endswith('_php_8kbp')]
ass_folders.sort()

In [11]:
# read in the gff files
p_gff = pd.read_csv(os.path.join(GENOME_PATH, P_GENOME+'.anno.gff3'), sep='\t', header=None)
h_gff = pd.read_csv(os.path.join(GENOME_PATH, H_GENOME+'.anno.gff3'), sep='\t', header=None)

In [12]:
def same_contig_blast(x,y):
    '''Function that checks if the blast hit in column y is on the same contig as the the query sequence in
    column y.
    '''
    try:
        q_contig = re.search(r'[p|h]([a-z]*_[0-9]*)_?', x).group(1)
        if y != 'NaN':
            hit_contig = re.search(r'[p|h]([a-z]*_[0-9]*)_?', y).group(1)
        else:
            hit_contig = 'NaN'
        return q_contig == hit_contig
    except AttributeError:
        print("AttributeError\nx: %s\ny: %s" % (x, y))
        sys.exit()
    except TypeError:
        print("x: %s\ny: %s" % (x, y))
        print("TypeError\ntype(x): %s\ntype(y): %s" % (type(x), type(y)))
        sys.exit()


In [13]:
#read in the blast df and add QLgth and QCov columns
blast_out_dict = {}
blast_header = ['Query', 'Target', 'PctID', 'AlnLgth', 'NumMis', 'NumGap', 'StartQuery', 'StopQuery', 'StartTarget',\
              'StopTarget', 'e-value','BitScore']
for key in blast_stderr_dict.keys():
    file_loc = key.split('>')[-1][1:]
    # file_loc = .../output/defining_alleles/allele_analysis/DK_0911_v03_h_ctg.DK_0911_v03_p_ctg.0.001.blastp.outfmt6
    
    file_name = file_loc.split('/')[-1]
    # DK_0911_v03_h_ctg.DK_0911_v03_p_ctg.0.001.blastp.outfmt6
    
    tmp_df = pd.read_csv(file_loc, sep='\t', header=None, names=blast_header)

    tmp_df['QLgth'] = tmp_df['Query'].apply(lambda x: length_dict[x])
    tmp_df['QCov'] = tmp_df['AlnLgth']/tmp_df['QLgth']*100
    
    tmp_df['TLgth'] = tmp_df['Target'].map(length_dict)
    tmp_df['TCov'] = tmp_df['AlnLgth']/tmp_df['TLgth']*100
    
    tmp_df.sort_values(by=['Query', 'e-value','BitScore', ],ascending=[True, True, False], inplace=True)
    
    # assert(len(tmp_df.loc[tmp_df['Query'] == 'pcontig_000:10023-10824']) == 0)
    
    #now make sure to add proteins/genes without blast hit to the DataFrame
    #check if gene blast or protein blast and pull out respective query file
    if file_loc.split('.')[-2] == 'blastp':
        tmp_blast_seq = [x for x in blast_dict['protein_fa'] if x.startswith(file_name.split('.')[0])][0]
    elif file_loc.split('.')[-2] == 'blastn':
        tmp_blast_seq = [x for x in blast_dict['gene_fa'] if x.startswith(file_name.split('.')[0])][0]
    tmp_all_blast_seq = []
    #make list of all query sequences
    for seq in SeqIO.parse(os.path.join(BLAST_DB, tmp_blast_seq), 'fasta'):
        tmp_all_blast_seq.append(str(seq.id))
    tmp_all_queries_w_hit = tmp_df["Query"].unique()
    tmp_queries_no_hit = set(tmp_all_blast_seq) - set(tmp_all_queries_w_hit)
    no_hit_list = []
    
    #loop over the quieres with no hit and make list of list out of them the first element being the query id
    for x in tmp_queries_no_hit:
        NA_list = ['NaN'] * len(tmp_df.columns)
        NA_list[0] = x
        no_hit_list.append(NA_list)
    
    tmp_no_hit_df = pd.DataFrame(no_hit_list)
    tmp_no_hit_df.columns = tmp_df.columns
    tmp_no_hit_df['QLgth'] = tmp_no_hit_df.Query.apply(lambda x: length_dict[x])
    #map stuff at the tmp level
    
    tmp_df = tmp_df.append(tmp_no_hit_df)
    assert(len(tmp_df.loc[tmp_df['Query'] == 'NaN']) == 0)
    
    tmp_df['q_contig'] = tmp_df['Query'].str.extract(r'([p|h][a-z]*_[^.]*).?', expand = False)
    tmp_df['t_contig'] = tmp_df['Target'].str.extract(r'([p|h][a-z]*_[^.]*).?', expand = False)
    # print(tmp_df['q_contig'])
    
    #fix that if you don't extract anything return False and not 'nan'
    tmp_df['t_contig'].fillna(False, inplace=True)
    
    # print(tmp_df.apply(lambda row: print(row['Query'])), axis = 1)
    
    tmp_df['q_contig == t_contig'] = tmp_df.apply(lambda row: same_contig_blast(row['Query'], row['Target']), axis = 1)
    tmp_df.reset_index(inplace=True, drop=True)
    blast_out_dict[file_name] = tmp_df.iloc[:,:]

In [14]:
for folder in ass_folders:
    '''
    From oriented_coords.csv files generate gff files with the following set up.
    h_contig overlap
    'query', 'ref', 'h_feature', 'query_start', 'query_end', 'query_aln_len', 'strand', 'frame', 'attribute_h'
    p_contig overlap
    'ref', 'query', 'p_feature','ref_start','ref_end', 'alignment_length', 'strand', 'frame', 'attribute_p'.
    Save those file in a new folder for further downstream analysis. The same folder should include the contig and gene filtered
    gffs. 
    '''
    orient_coords_file = [os.path.join(folder, x) for x in os.listdir(folder) if x.endswith('oriented_coords.csv')][0]
    #load in df and generate several additions columns
    tmp_df = pd.read_csv(orient_coords_file, sep=',')
    #check if there is any resonable alignment if not go to the next one
    if len(tmp_df) == 0:
        print('Check on %s assemblytics' % (folder))
        continue
    tmp_df['p_feature'] = "haplotig"
    tmp_df['h_feature'] ='primary_contig'
    tmp_df['strand'] = "+"
    tmp_df['frame'] = 0
    tmp_df['query_aln_len'] = abs(tmp_df['query_end']-tmp_df['query_start'])
    tmp_df['alignment_length'] = abs(tmp_df['ref_end'] - tmp_df['ref_start'])
    tmp_df.reset_index(drop=True, inplace=True)
    tmp_df.sort_values('query', inplace=True)
    tmp_df['attribute_p'] = tmp_df.apply(lambda row: att_column_p(row['query'],row['query_start'], row['query_end'],\
                                                         row['query_length'], row['query_aln_len']), axis=1)
    
    tmp_df['attribute_h'] = tmp_df.apply(lambda row: att_column_h(row['ref'], row['ref_start'], row['ref_end'],\
                                                                  row['ref_length'], row['alignment_length']), axis=1)
    
    #generate tmp gff dataframe
    tmp_p_gff_df = tmp_df.loc[:, ['ref', 'query', 'p_feature','ref_start','ref_end', 'alignment_length', 'strand', 'frame', 'attribute_p']]
    #now filter added if start < stop swap round. And if start == 0. This was mostly?! only? the case for haplotigs
    tmp_p_gff_df['comp'] = tmp_p_gff_df['ref_start'] - tmp_p_gff_df['ref_end']
    tmp_swap_index  = tmp_p_gff_df['comp'] > 0
    tmp_p_gff_df.loc[tmp_swap_index, 'ref_start'] , tmp_p_gff_df.loc[tmp_swap_index, 'ref_end'] = tmp_p_gff_df['ref_end'], tmp_p_gff_df['ref_start']
    #and if 'ref_start' == g
    is_null_index  = tmp_p_gff_df['ref_start'] == 0
    tmp_p_gff_df.loc[is_null_index, 'ref_start'] = 1
    tmp_p_gff_df = tmp_p_gff_df.iloc[:, 0:9]
    tmp_p_gff_df.sort_values('ref_start',inplace=True)
    tmp_h_gff_df = tmp_df.loc[:, ['query', 'ref', 'h_feature', 'query_start', 'query_end', 'query_aln_len', 'strand', 'frame', 'attribute_h']]
    #same fix for tmp_h_gff
    tmp_h_gff_df['comp'] = tmp_h_gff_df['query_start'] - tmp_h_gff_df['query_end']
    tmp_swap_index  = tmp_h_gff_df['comp'] > 0
    tmp_h_gff_df.loc[tmp_swap_index, 'query_start'] , tmp_h_gff_df.loc[tmp_swap_index, 'query_end'] = tmp_h_gff_df['query_end'], tmp_h_gff_df['query_start']
    #and if 'query_start' == g
    is_null_index  = tmp_h_gff_df['query_start'] == 0
    tmp_h_gff_df.loc[is_null_index, 'query_start'] = 1
    tmp_h_gff_df = tmp_h_gff_df.iloc[:, 0:9]
    tmp_h_gff_df.sort_values('query_start', inplace=True)
    #make the outfolder and save gff files of overlap and filtered gene files
    folder_suffix = folder.split('/')[-1]
    #get the contig
    contig = re.search(r'p([a-z]*_[0-9]*)_', folder_suffix).group(1)
    out_folder = os.path.join(OUT_PATH, folder_suffix)
    if not os.path.exists(out_folder):
        os.mkdir(out_folder)
    out_name_p = os.path.join(out_folder, folder_suffix )
    tmp_p_gff_df.to_csv(out_name_p+'.p_by_h_cov.gff', sep='\t' ,header=None, index =None)
    out_name_h =  os.path.join(out_folder, folder_suffix)
    tmp_h_gff_df.to_csv(out_name_h+'.h_by_p_cov.gff' , sep='\t' ,header=None, index =None)
    #write those out to new folder for php together with the p and h gff of genes
    p_gene_gff = gene_contig_subset(p_gff, contig, 'gene')
    p_gene_gff.to_csv(out_name_p+'.p_gene.gff', sep='\t' ,header=None, index =None)
    h_gene_gff = gene_contig_subset(h_gff, contig, 'gene')
    h_gene_gff.to_csv(out_name_h+'.h_gene.gff', sep='\t' ,header=None, index =None)
    
    #next would be to do overlap between the gene gff and the corresponding alignment gff. Write this out. Dict for each gene and its corresponding h_contig/p_contig
    

/home/benjamin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


Check on /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Assembletics/Pst_104E_v12_pcontig_118_php_8kbp assemblytics
Check on /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Assembletics/Pst_104E_v12_pcontig_193_php_8kbp assemblytics


In [15]:
p_gene_gff.head()

,0,1,2,3,4,5,6,7,8
0,pcontig_225,EVM,gene,2067,2309,.,+,.,ID=evm.TU.pcontig_225.1;Name=gene_model_pconti...
1,pcontig_225,EVM,gene,2658,2883,.,+,.,ID=evm.TU.pcontig_225.2;Name=gene_model_pconti...
2,pcontig_225,EVM,gene,3999,5386,.,+,.,ID=evm.TU.pcontig_225.3;Name=gene_model_pconti...
3,pcontig_225,EVM,gene,13375,15212,.,+,.,ID=evm.TU.pcontig_225.4;Name=gene_model_pconti...
4,pcontig_225,EVM,gene,17272,18147,.,+,.,ID=evm.TU.pcontig_225.5;Name=gene_model_pconti...


In [16]:
#now loop over the outfolders
out_folders = [os.path.join(OUT_PATH, x) for x in os.listdir(OUT_PATH) if x.endswith('_php_8kbp')]
out_folders.sort()

In [17]:
p_gene_h_contig_overlap_dict = {}
for out_folder in out_folders:
    tmp_folder_content = os.listdir(out_folder)
    tmp_p_gene_gff = [os.path.join(out_folder, x) for x in tmp_folder_content if x.endswith('p_gene.gff') ][0]
    tmp_p_gene_bed = pybedtools.BedTool(tmp_p_gene_gff).remove_invalid().saveas()
    tmp_p_by_h_gff = [os.path.join(out_folder, x) for x in tmp_folder_content if x.endswith('p_by_h_cov.gff') ][0]
    tmp_p_by_h_bed = pybedtools.BedTool(tmp_p_by_h_gff).remove_invalid().saveas()
    #generate a bed intersect
    p_gene_and_p_by_h = tmp_p_gene_bed.intersect(tmp_p_by_h_bed, wb=True)
    #check the length of the obtained intersect is 
    print("This is the length of the intersect %i" %len(p_gene_and_p_by_h ))
    if len(p_gene_and_p_by_h) == 0:
        continue
    #turn this into a dataframe 
    p_gene_and_p_by_h_df = p_gene_and_p_by_h.to_dataframe()
    p_gene_and_p_by_h_df['p_gene'] = p_gene_and_p_by_h_df[8].str.extract(r'ID=([^;]*);', expand=False)
    p_gene_and_p_by_h_df['p_protein'] = p_gene_and_p_by_h_df['p_gene'].str.replace('TU', 'model')
    #save this in the same folder as dataframe
    p_gene_and_p_by_h_df.to_csv(tmp_p_gene_gff.replace('p_gene.gff', 'gene_haplotig_intersect.df'), sep='\t', index=None)
    p_gene_and_p_by_h_df_2 = p_gene_and_p_by_h_df.loc[:, [10, 'p_protein']]
    #make a dict for the overlap of each gene with it's correspoding haplotig
    tmp_p_gene_h_overlap_df = p_gene_and_p_by_h_df_2.groupby('p_protein')[10].apply(list)
    tmp_p_gene_h_overlap_dict = dict(zip(tmp_p_gene_h_overlap_df.index, tmp_p_gene_h_overlap_df.values))
    p_gene_h_contig_overlap_dict.update(tmp_p_gene_h_overlap_dict)
    

This is the length of the intersect 588


/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3287: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))


This is the length of the intersect 548


/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3287: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))


This is the length of the intersect 496
This is the length of the intersect 417


...

/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3287: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))


This is the length of the intersect 5
This is the length of the intersect 0


/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3287: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))


This is the length of the intersect 6
This is the length of the intersect 9


/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3287: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))
/home/benjamin/anaconda3/lib/python3.5/site-packages/pybedtools/bedtool.py:3287: UserWarning: Default names for filetype gff are:
['seqname', 'source', 'feature', 'start', 'end', 'score', 'strand', 'frame', 'attributes']
but file has 18 fields; you can supply custom names with the `names` kwarg
  % (self.file_type, _names, self.field_count()))


In [18]:
def get_h_gene_p_contig_overlap_dict(p_gene_h_contig_overlap_dict):
    d = {}
    for key in p_gene_h_contig_overlap_dict.keys():
        for val in p_gene_h_contig_overlap_dict[key]:
            if val in d:
                d[val].append(key)
            else:
                d[val] = [key]
    return d
h_gene_p_contig_overlap_dict = get_h_gene_p_contig_overlap_dict(p_gene_h_contig_overlap_dict)

In [19]:
#get out the p_protein on h_protein blast out dataframe and appand the h_contig overlap for each protein
_key = [x for x in blast_out_dict.keys() if x.startswith(P_GENOME) and x.split('.')[-2] == 'blastp' and x.endswith('outfmt6')][0]
p_gene_h_contig_overlap_df = pd.DataFrame([p_gene_h_contig_overlap_dict.keys(), p_gene_h_contig_overlap_dict.values()],  index = ['p_protein', 'h_contig_overlap']).T
_tmp_df = blast_out_dict[_key]
_tmp_df = _tmp_df.merge(p_gene_h_contig_overlap_df, how='outer' ,left_on='Query', right_on='p_protein')
_tmp_df['h_contig_overlap'].fillna(False, inplace=True)
#check if the protein hit resites on the overlapping haplotig
_tmp_df['t_contig == h_contig_overlap'] = _tmp_df.apply(lambda row: target_on_mapped_haplotig(row['t_contig'], row['h_contig_overlap']), axis =1 )
#blast_out_dict[_key] = _tmp_df

In [20]:
#write the dataframe again
_tmp_df.to_csv(os.path.join(OUT_PATH, _key+'.allele_analysis'), index=None, sep='\t')

In [21]:
os.chdir('/home/benjamin/genome_assembly/Warrior/RT_script_base/DK_0911-1/scripts')
%run 'Pst_104E_proteinortho_RT.ipynb'

/home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/032017_assembly
Checking for correct files and directories...
Created directory: /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/allele_analysis/defining_alleles/proteinortho/ph_ctg_516
Copying Pst_104E_v12_h_ctg.anno.protein.faa file to /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/allele_analysis/defining_alleles/proteinortho/ph_ctg_516
Copying Pst_104E_v12_p_ctg.anno.protein.faa file to /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/allele_analysis/defining_alleles/proteinortho/ph_ctg_516
Writing a new proteinortho-compatible gff file at /home/benjamin/genome_assembly/PST79/FALCON/p_assemblies/v9_1/Pst_104E_v12/Warrior_comp_runs/allele_analysis/defining_alleles/proteinortho/ph_ctg_516/Pst_104E_v12_h_ctg.anno.protein.gff based on original at /home/benjamin/genome_assembly/PST79/FALCON/p_

b'[OUTPUT] -> written to ph_ctg_516.proteinortho'
b'Writing graph...'
b'Cleaning edge list...'
b'Removed 16 / 12228 edges'
b'Done.'
b'[OUTPUT] -> written to ph_ctg_516.proteinortho-graph'
b''
b'Clustering by gene-order (POFF mode)'
b'Reading ph_ctg_516.ffadj-graph'
b'2 species'
b'21723 paired proteins'
b'11007 bidirectional edges'
b'\r                          \rClustering: 0.01%\r                          \rClustering: 0.02%\r                          \rClustering: 0.03%\r                          \rClustering: 0.04%\r                          \rClustering: 0.05%\r                          \rClustering: 0.06%\r                          \rClustering: 0.06%\r                          \rClustering: 0.07%\r                          \rClustering: 0.08%\r                          \rClustering: 0.09%\r                          \rClustering: 0.10%\r                          \rClustering: 0.11%\r                          \rClustering: 0.12%\r                          \rClustering: 0.13%\r     

b'[OUTPUT] -> written to ph_ctg_516.poff'
b'Writing graph...'
b'Cleaning edge list...'
b'Removed 0 / 11007 edges'
b'Done.'
b'[OUTPUT] -> written to ph_ctg_516.poff-graph'
b''
b'All finished.'
Proteinortho finished running.


In [22]:
poff_graph_header = ['Target', 'Query', 'evalue_ab', 'bitscore_ab', 'evalue_ba', 'bitscore_ba', 'same_strand' , 'simscore']
po_df = pd.read_csv(poff_graph_fn, sep='\t', header=None, names=poff_graph_header, comment='#' )


In [23]:
len(po_df)

11007

In [24]:
def reduceGroups(g, labels, maxBest):
    '''returns the best hit based on e-value and BitScore per group'''
    for label, maxBest in zip(labels, maxBest):
        if len(g) == 1:
            return g
        if maxBest:
            g = g[g[label] == g[label].max()]
        else:
            g = g[g[label] == g[label].min()]
#     if len(g) > 1:
#         print('Could not reduce group to 1 element. %s ' % g['Query'])
    return g

In [25]:
# Add a comparision column to po_df and _tmp_df
po_df['comp'] = po_df['Query'] + po_df['Target']
_tmp_df['comp'] = _tmp_df['Query'] + _tmp_df['Target']

#here we need to sort on this new comp column and just keep the best hit
#this should fix the issue


#generate a new allele_source column
_tmp_df['allele_source'] = 'BLAST'
_tmp_df.loc[_tmp_df[_tmp_df.comp.isin(po_df.comp.unique())].index, 'allele_source' ] = 'PO'
#now drop the comp column again
_tmp_df = _tmp_df.drop('comp', 1)

print(_tmp_df.columns)

Index(['Query', 'Target', 'PctID', 'AlnLgth', 'NumMis', 'NumGap', 'StartQuery',
       'StopQuery', 'StartTarget', 'StopTarget', 'e-value', 'BitScore',
       'QLgth', 'QCov', 'TLgth', 'TCov', 'q_contig', 't_contig',
       'q_contig == t_contig', 'p_protein', 'h_contig_overlap',
       't_contig == h_contig_overlap', 'allele_source'],
      dtype='object')


In [26]:
_tmp_df.head()

,Query,Target,PctID,AlnLgth,NumMis,NumGap,StartQuery,StopQuery,StartTarget,StopTarget,...,QCov,TLgth,TCov,q_contig,t_contig,q_contig == t_contig,p_protein,h_contig_overlap,t_contig == h_contig_overlap,allele_source
0,evm.model.pcontig_000.10,evm.model.hcontig_000_031.5,70.19,161,1,1,1,114,1,161,...,141.228,161,100,pcontig_000,hcontig_000_031,True,evm.model.pcontig_000.10,"[hcontig_000_003, hcontig_000_031]",True,PO
1,evm.model.pcontig_000.10,evm.model.hcontig_000_003.11,59.68,124,3,1,1,77,1,124,...,108.772,219,56.621,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.10,"[hcontig_000_003, hcontig_000_031]",True,BLAST
2,evm.model.pcontig_000.100,evm.model.hcontig_000_003.107,99.21,1141,9,0,1,1141,1,1141,...,99.7378,1163,98.1083,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.100,[hcontig_000_003],True,PO
3,evm.model.pcontig_000.100,evm.model.hcontig_000_003.112,72.12,104,13,1,473,576,4,91,...,9.09091,122,85.2459,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.100,[hcontig_000_003],True,BLAST
4,evm.model.pcontig_000.101,evm.model.hcontig_000_003.108,89.03,237,20,1,1,231,1,237,...,102.597,237,100,pcontig_000,hcontig_000_003,True,evm.model.pcontig_000.101,[hcontig_000_003],True,PO


In [27]:
print(len(_tmp_df))
no_PO_df = _tmp_df[_tmp_df['allele_source'] == 'BLAST']
PO_df = _tmp_df[_tmp_df['allele_source'] == 'PO']
no_PO_df = no_PO_df.groupby('Query').apply(lambda g: reduceGroups(g, ['e-value', 'BitScore'], [False, True]))
_tmp_df = no_PO_df.append(PO_df, ignore_index=True)
#now remove hits of the dataframe where query and target are the same
_tmp_df.drop_duplicates(['Query', 'Target'], inplace=True)

284007


In [28]:
_tmp_df.to_csv(os.path.join(ALLELE_PATH, P_GENOME + '.full_df.alleles'), sep='\t', index=None)

In [29]:
htgOnPctgBlast = pd.read_csv(os.path.join(OUT_PATH, '%s.%s.0.001.blastp.outfmt6' % (H_GENOME, P_GENOME)), sep='\t', index_col=None, header=None, names=blast_header)
htgOnPctgBlast['QLgth'] = htgOnPctgBlast['Query'].apply(lambda x: length_dict[x])

## for the h on p BLAST, calculating QCov=AlnLgth/QLgth*100% provides a misleading figure,
## as the Query becomes the haplotig, which is shorter than the primary contig.
## Thus we also calculate: TCov=AlnLgth/TLgth*100% (i.e. alignment/(primary protein length))
htgOnPctgBlast['QCov'] = htgOnPctgBlast['AlnLgth']/htgOnPctgBlast['QLgth']*100
htgOnPctgBlast['TLgth'] = htgOnPctgBlast['Target'].map(length_dict)
htgOnPctgBlast['TCov'] = htgOnPctgBlast['AlnLgth']/htgOnPctgBlast['TLgth']*100

htgOnPctgBlast['q_contig'] = htgOnPctgBlast['Query'].str.extract(r'([p|h][a-z]*_[^.]*).?', expand = False)
htgOnPctgBlast['t_contig'] = htgOnPctgBlast['Target'].str.extract(r'([p|h][a-z]*_[^.]*).?', expand = False)

htgOnPctgBlast['q_contig == t_contig'] = htgOnPctgBlast.apply(lambda row: same_contig_blast(row['Query'], row['Target']), axis = 1)
htgOnPctgBlast.reset_index(inplace=True, drop=True)

#get out the p_protein on h_protein blast out dataframe and append the h_contig overlap for each protein
p_gene_h_contig_overlap_df = pd.DataFrame([h_gene_p_contig_overlap_dict.keys(), h_gene_p_contig_overlap_dict.values()],  index = ['p_protein', 'h_contig_overlap']).T

htgOnPctgBlast = htgOnPctgBlast.merge(p_gene_h_contig_overlap_df, how='outer' ,left_on='q_contig', right_on='p_protein')
htgOnPctgBlast['h_contig_overlap'].fillna(False, inplace=True)
# check if the protein hit resides on the overlapping haplotig

htgOnPctgBlast['t_contig == h_contig_overlap'] = htgOnPctgBlast.apply(lambda row: target_on_mapped_haplotig(row['Target'], row['h_contig_overlap']), axis = 1)
htgOnPctgBlast['allele_source'] = 'h_rBLAST'
htgOnPctgBlast = htgOnPctgBlast.groupby('Query').apply(lambda g: reduceGroups(g, ['e-value', 'BitScore'], [False, True]))

#now remove hits of the dataframe where query and target are the same

htgOnPctgBlast.drop_duplicates(['Query', 'Target'], inplace=True)

In [30]:
htgOnPctgBlast.to_csv(os.path.join(ALLELE_PATH, H_GENOME + '.full_df.alleles'), sep='\t', index=None, header=True)

In [31]:
for boolean in _tmp_df.columns == htgOnPctgBlast.columns:
    if boolean == False:
        raise Exception('Both these DataFrames must have the same column headers for This script to work.')

In [32]:
if _tmp_df[_tmp_df.allele_source == 'PO'].shape[0] == po_df.shape[0] == False:
    raise Exception('Please check the proteinortho input and the blast input.\
    There are an incorrect number of allele pairings in the final dataframe.')